![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidget_1 Humidity sensor

This short notebook introduces how we access a Phidget device from a Jupyter notebook.

It is important to run this notebook in the Google Chrome or Opera web browser, on a desktop or laptop computer. Unfortunately, other browsers (FireFox, Safari, Edge) will not work as they cannot access the USB port that connects to the Phidget device.


## Setting up the Phidget device

We will be using the Phidget Starter Kit, as shown in this figure:
<img src="./images/KIT4003.jpg" alt="Image of the starter kit"  width = 500 />

Verify these points:
- Get a hold of a Phidgets Starter Kit. It should already be assembled. If not, assemble it now.
    - You will have received instructions with the kit on how to assemble it.
- The Starter Kit has a humidity sensor attached. If it does not, find the sensor and attach it.
    - The humidity sensor is a small black box attached with a thin cable. It is marked with the initials "HUM."
- Connect the Starter Kit to your computer with a USB cable. 
    - The Kit come with a USB cable, it is attached to the Hub device inside the Starter Kit.
- If you have an older version of the Phidgets, you may need to update its firmware. See the following for details:
    - https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/
- As mentioned earlier, make sure you are running this notebook in the Google Chrome browser. 

### Notes on local and remote computer

Notice we are using two computers here. There is the laptop (or desktop) computer sitting in front of you, which is the **local** computer. The Callysto Hub, that you're running this notebook on, is actually a piece of software running on a cloud-based network computer located at the Cybera offices in Alberta (usually), and we call this the **remote** computer. 

We want to program in Python on the Callysto Hub, which will be running on the remote computer. The Phidget device is connected to the local computer (the one in front of you) using the USB port. So we need a way to communicate with the device using the local computer. 

We can do this using the Chrome (or Opera) browser and adding some code in the JavaScript language. JavaScript runs on the local computer, in the browser, and so it is able to access the Phidget device. This means we need to include some JavaScript in our notebook, as it has capabilities that Python does not. 

### A bit about JavaScript

JavaScript (JS) is just another programming language. It has its own syntax, and there are books and videos that will teach you about JavaScript. We don't want to teach it here, but will introduce some basic features so you can read the code in this Jupyter notebook. 

JS looks a lot like Python. Here are some key differences in JS:

- In JS, statements should end with a semicolon, as in: **y = 2+3;**
- In JS, variables need to be declared, as in: **var x = 0;**
- In JS, blocks of code are surrounded by braces, **{ }** like this:
```
if (do==true) {
    do_this;
    and_this;
} else {
    do_that;
}
```
- In JS, there is a way to try a piece of code, and if it fails, catch the error. This will be useful as when connecting the Phidget devices. If there is a problem with the hardware or connection, we can handle the error instead of just crashing the program. The syntax looks like this:
```
try {do_this;} catch {handle_error;}
```
- finally, there are many options to import code modules in JS. We will use the ESM format:
```
import {someFunction} from "https://someESMrepo.com/someFile";
```


## Overview of the code.

It's a bit tricky to use the Phidget device, as we have to "tell" the computer explicity that we want to use the Phidget. This means writing code that tells the computer that first, we want to use the USB port and cable to connect to the device; second, then we want to use a specific device in the starter kit; and finally we want to read data from that device. 

We will do this in a step-by-step manner, so we can understand how the connection process works. At the end, it is always good practice to clean things up by closing all the connections. 

The main steps are thus:

1. Import a Python library called "anywidget" that will hold the JS code.
2. In JS, import the Phidget code from the Phidget company's online repo:
```
    import {USBConnection, HumiditySensor} from "https://esm.sh/phidget22@3.17";
```
3. Open the USB connection and request access from this web page.
```
    connection = new USBConnection();
    connection.connect();
    connection.requestWebUSBDeviceAccess();
```
4. Open the Humidity device.
```
    channel = new HumiditySensor();
    channel.open();
```
5. Setup the device to save the Humidity data when the measured value changes.
```
    channel.onHumidityChange = function(humidity) {
       textBox = humidity;
    };

```
6. When we are done, close the device and USB connection
```
    channel.close();
    connection.close();
    connection.delete();
```

The details in the code below get complicated because we need to use the try/catch method to watch out for errors. We also use the "await" keyword to tell the computer to wait for a bit as it attempts to connect the Phidget. We add some logic to keep track of whether the device is connected or not, and to do the appropriate thing. 

## The user interface

Finally, we need a user interface. In the JS code, we will create a button and two text boxes. The button is to connect and disconnect the Phidget. The first text box will report the status of the connection. The second text box reports the data value read from the humidity sensor. 

The JS code connects the user interface to the functions that control the Phidgets. 

## Step 1.

Import the **anywidget** Python library. This code library is still in development, so we do a pip install if it if not already in the Jupyter hub. 

In [1]:
%pip install anywidget

In [2]:
import anywidget

## NOTE:

If the **install** or **import** above gives an error message, try running them aghain. If that doesn't work, try closing the notebook, shut down the kernel, then reopen the notebook. Run the cell again. After that, the library will be properly installed.  

## Steps 2 through 5.

We put all the JS code into an **anywidget** class that we name "PhidgetHum."

The first section labelled **\_esm** contains the code to control the Phidget, as well as creating the user interface.

The second section labelled **\_css** is a collection of style choices for our button. 



In [3]:
class PhidgetHum(anywidget.AnyWidget):
  _esm = """

// Step 2. Importing the functions we need for Phidgets
  import {USBConnection, HumiditySensor} from "https://esm.sh/phidget22@3.17";

  export function render({ model, el }) {
    
    var connection = 0, channel = 0, connOpen = false, chOpen = false, isOpen = false;

// Steps 3 and 4. Opening the USBconnection and humidity device
    async function openUSB() {
        if (connection) {
            try {await channel.close();} catch {}
            try {await connection.close();} catch {}
            try {await connection.delete();} catch {}
        }
        try {connection = new USBConnection();} 
            catch{console.log("This browser does not support WebUSB");}
        channel = new HumiditySensor();
        channel.onAttach = (ch) => {
            ch.setDataInterval(1000);
        }

// Step 5. Reading and saving the humidity data
        channel.onHumidityChange = function(humidity) {
            textHumid.innerHTML = '  Humidity is ' + humidity;
       	};
    	try {
    		await connection.connect();
            connection.requestWebUSBDeviceAccess();
            connOpen = true;
    	} catch(err) {
            connOpen = false;
    	}
        if (connOpen) {
            chOpen = false;
            try {
                await channel.open(2000);
                chOpen = true;
            } catch(err) {
                chOpen = false;                
            }
        }
        
         // now get the UI to reflect the status
        if (connOpen && chOpen) {
            isOpen = true;
            textStatus.innerHTML = 'USB connected, humidity channel open.  ';
            button.innerHTML = `Click to disconnect`;
        }
        if (connOpen && !chOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB connected, channel not open.  ';
            button.innerHTML = `Click to connect`;
        }
        if (!connOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB did not connect.  ';
            button.innerHTML = `Click to connect`;        
        }
    };

// Step 6. Closing the Phidget.
    async function closeUSB() {
        try {await channel.close();} catch {}
        try {await connection.close();} catch {}
        try {await connection.delete();} catch {}
        isOpen = false; 
        connOpen = false;
        chOpen = false;
        textStatus.innerHTML = 'Disconnected.  ';
        textHumid.innerHTML = '  Humidity is null.';
        button.innerHTML = `Click to connect`;
    };
    
// The user interface, with a button and two text boxes
    let button = document.createElement("button");
    button.classList.add("ph-button");
    button.innerHTML = `Click to open USB`;
    button.addEventListener("click", async () => {
        if (isOpen) {closeUSB();} else {openUSB();}
    });
    let textStatus = document.createElement("label");
    textStatus.innerHTML = 'Status message here.  ';
    let textHumid = document.createElement("label");
    textHumid.innerHTML = 'Humidity is null.';
    
// Now display the user interface
 
    el.appendChild(button);
    el.appendChild(textStatus);
    el.appendChild(textHumid);
    
// Step 6, again. Clean up when the notebook is closed
    return async () => {
        try {await channel.close();} catch {}
        try {await connection.close();} catch {}
        try {await connection.delete();} catch {}
    };
  }
  """
  _css = """
        .ph-button {color: white; 
            background-color:rgb(96, 107, 174); 
            border-radius: 8px; 
            font-size: 24px; 
            display: block;
            padding: 15px 32px;}
        .ph-button:hover { background-color:rgb(120, 128, 187); }
  """

ph = PhidgetHum()

ph

PhidgetHum()

Clicking on the "Connect" button above will open a window that asks you to select the Phidget Hub. Select the hub and click "Connect." The window looks like this:
<img src="images/Pconnect.png" alt="Hey"  width = 300 />

If you don't get this window, perhaps you are using the wrong browser. (Use Chrome, Chromium, or Opera).

If you get the window but don't see the Phidget device, check to be sure the Phidget VINT hub is connected to your computer with a USB cable. 

### Check:
If all is well, the status text will report that the USB port is connected and the humidity channel is open. You should also see the value for humidity, which is reported as a percentance of relative humidity. 

If you don't see this, check again that the Phidget VINT device is connected to your computer with a USB cable. Also check that you have a Phidget humidity sensor connected to your VINT with the appropriate cable. 

## Step 6. Closing. 

Once we are all done, we should close the connection to the Phidget. Just click on the button (which should say "Close to disconnect") and the Phidget will be released.

Closing the notebook will also automatically release the Phidget.

Once the Phidget is released, it can be used by other notebooks or other software on your computer. 



### Confirm

You can confirm the Phidget is open or closed by looking at the status text beside the "Connect/Disconnect" button. 

## In case of errors

What kind of errors can happen? What should they do about them?

Symptom 1. When you click the connect button, status text says "USB not connected"
- Problem is likely that your browser does not support WebUSB.
    - Solution is to switch to Chrome, Chromium, Opera or Edge.
- Another problem could be that the Phidget VINT device is not connected via a USB cable. You should see a window like this <img src="images/Pconnect.png" alt="Hey"  width = 300 /> Select the VINT device once it appears.
- Another notebook or piece of software is already connected to the Phidget device.
    - Only one notebook at a time can use the Phidget. So close down the other notebook, or exit any other software using the device. 
- You might have an older version of a Phidget that needs a firmware update.
    - see here for the solution: https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/
  
    
Symptom 2. When you click the connect button, status text is "USB connected, channel not open."
- Check to see that you have a Phidget humidity device connected to your Phidget hub or VINT device


[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)